In [1]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
import numpy as np
from model import VAE
from train_eval_vae import to_numpy
from main import extract_pairs_for_development, PairDataGenerator
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)
dataset_path = '../lfw-deepfunneled/'

BATCH_SIZE = 16
g = torch.Generator()
g.manual_seed(SEED)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/rene/miniconda3/envs/tscl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
vae = VAE((3, 128, 128), 256, 3).to(device)
vae.load_state_dict(torch.load('trained-final/devFinal-(3, 128, 128)-256-3-0.1.dat')['state_dict'])
vae.eval()

VAE(
  (encoder_conv): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.01)
  )
  (mu): Sequential(
    (0): Linear(in_features=32768, out_features=256, bias=True)
    (1): Dropout(p=0.2, inplace=False)
  )
  (log_var): Sequential(
    (0): Linear(in_features=32768, out_features=256, bias=True)
    (1): Dropout(p=0.2, inplace=False)
  )
  (decoder_linear): Sequential(
    (0): Linear(in_features=256, out_features=32768, bias

In [3]:
pairs_train_path = '../pairsDevTrain.txt'
pairs_test_path = '../pairsDevTest.txt'
pairs_train = extract_pairs_for_development(pairs_train_path)
pairs_test = extract_pairs_for_development(pairs_test_path)

transform = transforms.Resize((128,128))
train_dataset = PairDataGenerator(pairs=pairs_train, transform=transform)
test_dataset = PairDataGenerator(pairs=pairs_test, transform=transform)

BATCH_SIZE = 32
train_pair_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, generator=g)
test_pair_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, generator=g)

train_left, train_right, train_values = to_numpy(train_pair_loader, vae)
test_left, test_right, y_test = to_numpy(test_pair_loader, vae)

In [4]:
train1 = np.c_[train_left, train_right]
train2 = np.c_[train_right, train_left]
X_train = np.r_[train1, train2]
y_train = np.r_[train_values, train_values]

X_test = np.c_[test_left, test_right]

In [5]:
sgd = SGDClassifier(random_state=SEED)
sgd.fit(X_train, y_train)

sgd_pred = sgd.predict(X_train)
sgd_acc = accuracy_score(y_train, sgd_pred)
print(f"SGD Train Accuracy: {sgd_acc}")

sgd_pred = sgd.predict(X_test)
sgd_acc = accuracy_score(y_test, sgd_pred)
print(f"SGD Test Accuracy: {sgd_acc}")

SGD Train Accuracy: 0.5802272727272727
SGD Test Accuracy: 0.516


In [6]:
svc = SVC(random_state=SEED)
svc.fit(X_train, y_train)

svc_pred = svc.predict(X_train)
svc_acc = accuracy_score(y_train, svc_pred)
print(f"SGD Train Accuracy: {svc_acc}")


svc_pred = svc.predict(X_test)
svc_acc = accuracy_score(y_test, svc_pred)
print(f"SVC Test Accuracy: {svc_acc}")

SGD Train Accuracy: 0.8640909090909091
SVC Test Accuracy: 0.616


In [7]:
rfc = RandomForestClassifier(random_state=SEED)
rfc.fit(X_train, y_train)

rfc_pred = rfc.predict(X_train)
rfc_acc = accuracy_score(y_train, rfc_pred)
print(f"RFC Train Accuracy: {rfc_acc}")

rfc_pred = rfc.predict(X_test)
rfc_acc = accuracy_score(y_test, rfc_pred)
print(f"RFC Test Accuracy: {rfc_acc}")

RFC Train Accuracy: 1.0
RFC Test Accuracy: 0.565


In [8]:
mlp = MLPClassifier(random_state=SEED)
mlp.fit(X_train, y_train)

mlp_pred = mlp.predict(X_train)
mlp_acc = accuracy_score(y_train, mlp_pred)
print(f"MLP Train Accuracy: {mlp_acc}")

mlp_pred = mlp.predict(X_test)
mlp_acc = accuracy_score(y_test, mlp_pred)
print(f"MLP Test Accuracy: {mlp_acc}")

MLP Train Accuracy: 1.0
MLP Test Accuracy: 0.619
